### **MACHINE LEARNING**
 Training and testing historical data to make prediction
 
 Core logic Mainly use XGBoost( eXtreme Gradient Boosting) framework


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from datetime import timedelta
from sklearn.metrics import mean_absolute_error


#load data and clean data 
def loaddata(filename):
    print(f'laoding data from {filename}')
    daily_df = pd.read_excel(filename, skiprows=3) ##skip the first 3 role since it just some info and latitute + longtitute
    
    #fixing Excel date data
    if pd.api.types.is_numeric_dtype(daily_df['time']):
        daily_df['time'] = pd.to_datetime(daily_df['time'], unit='D', origin='1899-12-30')
    daily_df = daily_df.set_index('time').sort_index()
    print(f"Loaded {len(daily_df)} days up to {daily_df.index[-1].date()}")

def cleandata(filename):
    

In [26]:
loaddata('test3.xlsx')

laoding data from test3.xlsx
Loaded 326 days up to 2025-11-22
